In [1]:
# import tensorflow as tf
import numpy as np
import pandas as pd
import transformers
from datasets import Dataset 
import torch
print(torch.__version__)
# print(tensorflow.__version__)
print(np.__version__)
print(transformers.__version__)
# print(datasets.__version__)

1.9.1+cu111
1.21.4
4.11.3


In [2]:
# !pip install torch==2.3.0 --index-url https://download.pytorch.org/whl/cu118

import torch
print(torch.__version__)



1.9.1+cu111


In [8]:
# !pip install transformers==4.37.0

  Using cached transformers-4.37.0-py3-none-any.whl (8.4 MB)
  Using cached tokenizers-0.15.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.3
    Uninstalling transformers-4.52.3:
      Successfully uninstalled transformers-4.52.3


In [9]:
# import os
# os._exit(0)

In [3]:
# import torch_xla
# import torch_xla.core.xla_model as xm
import transformers
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer


In [7]:
# !pip uninstall torch transformers -y
# !pip install torch==2.3.0 --index-url https://download.pytorch.org/whl/cu118


In [8]:
# !pip install transformers==4.40.1


# NSMC 데이터 분석 및 huggingface dataset 구성
- Naver Snetiment Movie Corpus task
- klue/ber-base를 활용하여 NSMC task를 하는 것이 목표

In [9]:
# 데이터 다운로드
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

# pandas로 데이터 읽기
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

# 결측값 제거
train_data = train_data.dropna(how='any').reset_index(drop=True)
test_data = test_data.dropna(how='any').reset_index(drop=True)

# Hugging Face Dataset으로 변환
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# 데이터셋 확인
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 149995
})
Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 49997
})


In [10]:
# 데이터셋을 dict 형식으로 변환
train_dict = train_dataset.to_dict('list')
test_dict = test_dataset.to_dict('list')

# dict로 변환하여 Hugging Face Dataset으로 다시 변환
hf_train_dataset = Dataset.from_dict(train_dict)
hf_test_dataset = Dataset.from_dict(test_dict)

# train/validation split (임의로 20% validation 데이터로 사용)
train_val_dataset = hf_train_dataset.train_test_split(test_size=0.2)
tf_train_dataset = train_val_dataset['train']
tf_val_dataset = train_val_dataset['test']
tf_test_dataset = hf_test_dataset  # 테스트 데이터

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')


/opt/conda/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
# tokenizer와 model
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

huggingface_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
huggingface_model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels = 2)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# !pip install transformers==4.37.0

# 불러온 tokenizer로 데이터셋 전처리, model 학습 진행

In [13]:
# import torch
# import numpy as np
# from transformers import AutoTokenizer
# from datasets import Dataset

# # 토크나이저 로드
# tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

# def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
#     encodings = tokenizer(
#         examples,  # 문서 리스트
#         truncation=True,  # 길이가 max_seq_len을 넘으면 자르기
#         padding='max_length',  # 길이가 max_seq_len 미만이면 패딩
#         max_length=max_seq_len,  # 최대 길이 설정
#         return_tensors=None,  # 텐서를 반환하지 않음, 대신 리스트나 numpy 배열 반환
#         return_attention_mask=True,  # Attention Mask 반환
#         return_token_type_ids=False  # 단일 문장 분류이므로 token_type_ids 필요 없음
#     )
    
#     # 토크나이즈된 값들
#     input_ids = encodings['input_ids']
#     attention_masks = encodings['attention_mask']
    
#     # 레이블을 리스트로 변환
#     data_labels = labels  # 이미 리스트 형태로 존재

#     # 딕셔너리 형태로 반환, 텐서 대신 리스트로 반환
#     return {
#         'input_ids': input_ids,
#         'attention_mask': attention_masks,
#         'labels': data_labels
#     }

# # 예시 데이터
# train_data = ["이 영화 진짜 재밌어요", "영화 내용이 지루했어요"]  # 예시 문장
# train_labels = [1, 0]  # 예시 레이블: 1 = 긍정, 0 = 부정

# # 데이터셋 변환
# max_seq_len = 128  # 최대 시퀀스 길이 설정
# train_dataset = Dataset.from_dict({'document': train_data, 'label': train_labels})

# # 데이터셋에 전처리 적용
# train_dataset = train_dataset.map(lambda x: convert_examples_to_features(x['document'], x['label'], max_seq_len, tokenizer), batched=True)

# # 전처리된 데이터셋 확인
# print(train_dataset[0])  # 첫 번째 샘플을 확인


In [14]:
from datasets import Dataset
from transformers import AutoTokenizer
import torch

import torch
from datasets import Dataset
from transformers import AutoTokenizer

# 예시 데이터
train_data = ["이 영화 진짜 재밌어요", "영화 내용이 지루했어요"]  # 예시 문장
train_labels = [1, 0]  # 예시 레이블: 1 = 긍정, 0 = 부정

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

# 전처리 함수 정의
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    # 토크나이저를 사용하여 데이터 배치 처리
    encodings = tokenizer(
        examples,  # 문서 리스트
        truncation=True,  # 길이가 max_seq_len을 넘으면 자르기
        padding='max_length',  # 길이가 max_seq_len 미만이면 패딩
        max_length=max_seq_len,  # 최대 길이 설정
        return_tensors=None,  # 텐서를 반환하지 않고, 리스트나 numpy 배열로 반환
        return_attention_mask=True,  # Attention Mask 반환
        return_token_type_ids=False  # 단일 문장 분류이므로 token_type_ids 필요 없음
    )
    
    # tokenized된 값들
    input_ids = encodings['input_ids']
    attention_masks = encodings['attention_mask']
    
    # 레이블을 텐서로 변환
    data_labels = labels  # 이미 리스트 형태로 존재

    # 딕셔너리 형태로 반환
    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'labels': data_labels
    }

# 데이터셋 전처리
max_seq_len = 128  # 최대 시퀀스 길이 설정

# tf_train_dataset, tf_val_dataset, tf_test_dataset에 전처리 적용
tf_train_dataset = tf_train_dataset.map(
    lambda x: convert_examples_to_features(x['document'], x['label'], max_seq_len, tokenizer),
    batched=True
)

tf_val_dataset = tf_val_dataset.map(
    lambda x: convert_examples_to_features(x['document'], x['label'], max_seq_len, tokenizer),
    batched=True
)

tf_test_dataset = tf_test_dataset.map(
    lambda x: convert_examples_to_features(x['document'], x['label'], max_seq_len, tokenizer),
    batched=True
)

# 전처리된 데이터셋 확인 (첫 번째 샘플 확인)
print(tf_train_dataset[0])  # 첫 번째 샘플을 확인

# 텐서로 변환
train_input_ids = torch.tensor(tf_train_dataset['input_ids'])
train_attention_mask = torch.tensor(tf_train_dataset['attention_mask'])
train_labels = torch.tensor(tf_train_dataset['labels'])

val_input_ids = torch.tensor(tf_val_dataset['input_ids'])
val_attention_mask = torch.tensor(tf_val_dataset['attention_mask'])
val_labels = torch.tensor(tf_val_dataset['labels'])

test_input_ids = torch.tensor(tf_test_dataset['input_ids'])
test_attention_mask = torch.tensor(tf_test_dataset['attention_mask'])
test_labels = torch.tensor(tf_test_dataset['labels'])

# 텐서 형태로 잘 변환되었는지 확인
print(train_input_ids.shape, train_attention_mask.shape, train_labels.shape)
print(val_input_ids.shape, val_attention_mask.shape, val_labels.shape)
print(test_input_ids.shape, test_attention_mask.shape, test_labels.shape)

  0%|          | 0/120 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

{'id': 9822864, 'document': '대서사시의 완벽하고 화려한 막', 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_ids': [2, 18979, 2063, 2067, 2079, 5537, 19521, 5725, 2470, 1037, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 1, 'label': 1}
torch.Size([119996, 128]) torch.Size([119996, 128]) torch.Size([119996])
torch.Size([29999, 128

In [15]:
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

# train_data = pd.read_table('ratings_train.txt')
# test_data = pd.read_table('ratings_test.txt')

# train_data = train_data.dropna(how = 'any')
# train_data = train_data.reset_index(drop=True)
# print(train_data.isnull().values.any())

# test_data = test_data.dropna(how = 'any')
# test_data = test_data.reset_index(drop=True)
# print(test_data.isnull().values.any())

In [16]:
# tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

# def convert_examples_to_features(examples,labels,max_seq_len,tokenizer):

#   input_ids = []
#   attention_masks = []
#   token_type_ids = []
#   data_labels = []

#   for example, label in tqdm(zip(examples,labels),total=len(examples)):

#     input_id = tokenizer.encode(example,max_length=max_seq_len,pad_to_max_length=True)
#     padding_count = input_id.count(tokenizer.pad_token_id)
#     attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
#     token_type_id = [0] * max_seq_len

#     assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
#     assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
#     assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

#     input_ids.append(input_id)
#     attention_masks.append(attention_mask)
#     token_type_ids.append(token_type_id)
#     data_labels.append(label)

#   input_ids = torch.tensor(input_ids)
#   attention_masks = torch.tensor(attention_masks)
#   token_type_ids = torch.tensor(token_type_ids)

#   data_labels = torch.tensor(data_labels)

#   return (input_ids, attention_masks, token_type_ids), data_labels

# max_seq_len = 128

# train_X,train_y = convert_examples_to_features(tf_train_dataset["document"],tf_train_dataset["label"],max_seq_len = max_seq_len, tokenizer = tokenizer)
# val_X, val_y = convert_examples_to_features(tf_val_dataset["document"],tf_val_dataset["label"],max_seq_len = max_seq_len,tokenizer=tokenizer)
# test_X, test_y = convert_examples_to_features(tf_test_dataset["document"],tf_test_dataset["label"],max_seq_len = max_seq_len,tokenizer=tokenizer)



# from tqdm import tqdm
# import torch

# def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
#     input_ids = []
#     attention_masks = []
#     data_labels = []

#     # 토크나이저를 사용하여 데이터 배치 처리
#     encodings = tokenizer(
#         examples,  # 문서 리스트
#         truncation=True,  # 길이가 max_seq_len을 넘으면 자르기
#         padding='max_length',  # 길이가 max_seq_len 미만이면 패딩
#         max_length=max_seq_len,  # 최대 길이 설정
#         return_tensors='pt',  # 텐서로 반환 (PyTorch)
#         return_attention_mask=True,  # Attention Mask 반환
#         return_token_type_ids=False  # 단일 문장 분류이므로 token_type_ids 필요 없음
#     )

#     # input_ids, attention_mask, labels를 리스트에 추가
#     input_ids = encodings['input_ids']
#     attention_masks = encodings['attention_mask']
#     data_labels = torch.tensor(labels)  # 레이블을 텐서로 변환

#     return (input_ids, attention_masks), data_labels

# # 최대 시퀀스 길이 설정
# max_seq_len = 128

# # 전처리 실행
# train_X, train_y = convert_examples_to_features(
#     tf_train_dataset["document"], 
#     tf_train_dataset["label"], 
#     max_seq_len=max_seq_len, 
#     tokenizer=huggingface_tokenizer
# )
# val_X, val_y = convert_examples_to_features(
#     tf_val_dataset["document"], 
#     tf_val_dataset["label"], 
#     max_seq_len=max_seq_len,
#     tokenizer=huggingface_tokenizer
# )
# test_X, test_y = convert_examples_to_features(
#     tf_test_dataset["document"], 
#     tf_test_dataset["label"], 
#     max_seq_len=max_seq_len, 
#     tokenizer=huggingface_tokenizer
# )


In [17]:
# import torch

# def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
#     # 입력값을 담을 리스트 초기화
#     input_ids = []
#     attention_masks = []
#     data_labels = []
    
#     # 토크나이저를 사용하여 데이터 배치 처리
#     encodings = tokenizer(
#         examples,  # 문서 리스트
#         truncation=True,  # 길이가 max_seq_len을 넘으면 자르기
#         padding='max_length',  # 길이가 max_seq_len 미만이면 패딩
#         max_length=max_seq_len,  # 최대 길이 설정
#         return_tensors='pt',  # 텐서로 반환 (PyTorch)
#         return_attention_mask=True,  # Attention Mask 반환
#         return_token_type_ids=False  # 단일 문장 분류이므로 token_type_ids 필요 없음
#     )
    
#     # tokenized된 값들
#     input_ids = encodings['input_ids']
#     attention_masks = encodings['attention_mask']
    
#     # 레이블을 텐서로 변환
#     data_labels = torch.tensor(labels)  # 레이블을 텐서로 변환
    
#     # 딕셔너리 형태로 반환
#     return {
#         'input_ids': input_ids,
#         'attention_mask': attention_masks,
#         'labels': data_labels
#     }

# # 예시 데이터
# max_seq_len = 128  # 최대 시퀀스 길이
# train_data = ["이 영화 진짜 재밌어요", "영화 내용이 지루했어요"]  # 예시 문장
# train_labels = [1, 0]  # 예시 레이블: 1 = 긍정, 0 = 부정

# # 토크나이저 로드
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

# # 전처리 함수 호출
# features = convert_examples_to_features(train_data, train_labels, max_seq_len, tokenizer)

# # 전처리된 데이터 출력
# print(features)


In [18]:
# !pip install accelerate -U
# !pip install transformers[torch]


In [19]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments
import transformers
print(transformers.__version__)
!pip install --upgrade transformers

output_dir = './transformers2'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)


4.40.1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.52.3-py3-none-any.whl (10.5 MB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
from datasets import load_metric
metric_accuracy = load_metric("accuracy")
metric_f1 = load_metric("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)  # 예측값을 라벨로 변환 (가장 높은 확률의 클래스 선택)
    
    # 정확도와 F1-score 계산
    accuracy = metric_accuracy.compute(predictions=predictions, references=labels)
    f1 = metric_f1.compute(predictions=predictions, references=labels)
    
    return {"accuracy": accuracy["accuracy"], "f1": f1["f1"]}

In [21]:
# import transformers
# print(transformers.__version__)


In [22]:
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=tf_train_dataset,    # training dataset
    eval_dataset=tf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()
print("슝~")

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.294000,0.286232,0.897997,0.897563
2,0.222200,0.374714,0.904863,0.903854
3,0.163600,0.458725,0.903263,0.903150


슝~


- accuracy :90%이상
- f1-score 90% 이상
- training loss는 꾸준히 감소하고 있다. 0.2940 -> 0.1630
- validation loss는 증가. 0.2862 -> 0.4487
- Accuracy / F1 Score는 약간 증가 후 약간 하락

    Accuracy:

        Epoch 1 → 2: 상승 (0.8979 → 0.9048)

        Epoch 2 → 3: 하락 (0.9048 → 0.9032)

    F1 Score:

        Epoch 1 → 2: 상승 (0.8975 → 0.9038)

        Epoch 2 → 3: 하락 (0.9038 → 0.9031)

→ 성능이 정체된 상태이며, 과적합 의심


In [23]:
# # 마지막 저장된 체크포인트를 로드 (예: checkpoint-5000)
# model = BertForSequenceClassification.from_pretrained('./transformers/checkpoint-5000')


# 학습 완료 후 모델 저장
trainer.save_model('./basemodel2')

In [24]:
trainer.evaluate(tf_test_dataset)

{'eval_loss': 0.46605202555656433,
 'eval_accuracy': 0.9026541592495549,
 'eval_f1': 0.9044017992182436,
 'eval_runtime': 508.4451,
 'eval_samples_per_second': 98.333,
 'eval_steps_per_second': 12.292,
 'epoch': 3.0}

- Accuracy/F1 90% 이상으로 우수
- eval loss 0.46으로 약간 높은 편, 약간의 overfitting
- validation과 test에서 모두 우수한 성능을 보임

# fine-tuning을 통하여 모델 accuracy 향상시키기
- early stopping 적용
- learning rate scheduler 사용
- learning rate 감소
- dropout 조정
- mixed precision traing  fp16 = true로 학습속도 올리기

In [51]:
#메모리 초기화
import torch, gc

def reset_cuda():
    gc.collect()
    torch.cuda.empty_cache()
    print("✅ GPU memory cleared")

reset_cuda()



✅ GPU memory cleared


In [52]:
from transformers import EarlyStoppingCallback

In [53]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained('klue/bert-base', num_labels=2)
config.hidden_dropout_prob = 0.3  # 기본은 0.1 → 과적합 방지를 위해 증가

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'klue/bert-base', 
    config=config
)


/opt/conda/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./fine_tune_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size =16,   # 각 device 당 batch size 8 -> 16로 
    per_device_eval_batch_size = 16,  
    learning_rate=1e-5,  
    lr_scheduler_type = 'cosine',                       #cosine scheduler 추가
    warmup_ratio = 0.1,                            #전체 step의 10%
    num_train_epochs=10,                          # 💡 EarlyStopping이 있으므로 더 크게 설정 가능
    weight_decay=0.01,                           # 일반화 성능 향상 l2 legularization
    load_best_model_at_end=True,                 # ✅ EarlyStopping과 함께 사용
    metric_for_best_model="accuracy",           # 조기 종료 기준 메트릭
    greater_is_better=True,                     # 높을수록 좋음 (accuracy의 경우)
    logging_dir="./logs",
    logging_strategy="epoch",
    fp16=True     #학습 속도 높이기, 메모리 사용 줄이기
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tf_train_dataset,
    eval_dataset=tf_val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # ✅ patience: 2 epoch
)
trainer.train()
trainer.save_model('./fine_tuned_model')
trainer.evaluate(tf_test_dataset)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.389800,0.276043,0.889363,0.888874
2,0.280700,0.283571,0.897463,0.897936
3,0.240500,0.261236,0.904430,0.903295
4,0.215100,0.331682,0.902430,0.903415


- train/val loss가 안정적으로 줄어든다. 
- accuracy/f1이 90% 이상으로 우수
- epoch 3 이후 accuracy가 떨어지고 train/val loss 증가
- epoch 수는 3을 넘지 않는 것이 좋아보임

# Bucketing을 적용하여 학습시키고 이전 모델(basemodel)과 비교
- 패딩을 dynamic으로 바꾼다음에 bucketing 적용
- 텍스트 길이에 따라 여러개의 bucket으로 나눠서 비슷한 길이끼리 같은 batch 구성
- 모델 성능 향상 측면
- 훈련 시간 측면

In [4]:
# !pip install --upgrade transformers

In [102]:
#메모리 초기화
import torch, gc

def reset_cuda():
    gc.collect()
    torch.cuda.empty_cache()
    print("✅ GPU memory cleared")

reset_cuda()

✅ GPU memory cleared


In [103]:
import pandas as pd
from datasets import Dataset

# 데이터 다운로드
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", "ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", "ratings_test.txt")

# pandas 로드 및 전처리
train_df = pd.read_table("ratings_train.txt").dropna().reset_index(drop=True)
test_df = pd.read_table("ratings_test.txt").dropna().reset_index(drop=True)

# HuggingFace Dataset 변환
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [104]:
# dynamic padding 적용
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

# tokenizer 함수 정의
def tokenize_function(examples):
    return tokenizer(
        examples["document"],
        truncation=True,
        padding=False,  # ✅ dynamic padding을 위해 False
        return_attention_mask=True
    )

# tokenize 적용
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [105]:
def add_length(example):
    example["length"] = sum(example["attention_mask"])
    return example

train_dataset = train_dataset.map(add_length)
test_dataset = test_dataset.map(add_length)

# 정렬
train_dataset = train_dataset.sort("length")
test_dataset = test_dataset.sort("length")



  0%|          | 0/149995 [00:00<?, ?ex/s]

  0%|          | 0/49997 [00:00<?, ?ex/s]

In [106]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from torch.utils.data.sampler import Sampler
import numpy as np


In [107]:
class BucketBatchSampler(Sampler):
    def __init__(self, lengths, batch_size):
        self.batch_size = batch_size
        self.sorted_indices = np.argsort(lengths)
        self.batches = [
            self.sorted_indices[i:i + batch_size]
            for i in range(0, len(self.sorted_indices), batch_size)
        ]

    def __iter__(self):
        np.random.shuffle(self.batches)
        for batch in self.batches:
            yield batch.tolist()

    def __len__(self):
        return len(self.batches)


In [108]:
from torch.utils.data import Dataset

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset):
        self.dataset = hf_dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return {
            "input_ids": self.dataset[idx]["input_ids"],           # ✅ 리스트 그대로 반환
            "attention_mask": self.dataset[idx]["attention_mask"],
            "labels": self.dataset[idx]["label"],                  # ❗ 'label' → 'labels'로 이름만 통일
        }



In [109]:
# train_dataset = train_dataset.rename_column("label", "labels")
# test_dataset = test_dataset.rename_column("label", "labels")

In [110]:
# dataloader 생성

from transformers import DataCollatorWithPadding

# Dataset 생성
train_data = CustomDataset(train_dataset)
test_data = CustomDataset(test_dataset)

# BucketBatchSampler
train_sampler = BucketBatchSampler(train_dataset["length"], batch_size=16)
test_sampler = BucketBatchSampler(test_dataset["length"], batch_size=16)

# Collator: 동적 패딩 적용
collator = DataCollatorWithPadding(tokenizer=tokenizer)

# DataLoader
train_loader = DataLoader(train_data, batch_sampler=train_sampler, collate_fn=collator)
test_loader = DataLoader(test_data, batch_sampler=test_sampler, collate_fn=collator)


In [111]:
# !pip install --upgrade transformers


In [112]:
import transformers
print(transformers.__version__)


4.11.3


In [113]:
from transformers import BertForSequenceClassification, AutoTokenizer

In [114]:
from transformers import BertForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
import time

# ✅ device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 모델 로드
model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2).to(device)

# ✅ Optimizer 설정 (Trainer와 동일하게 설정)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)



# ✅ 평가 지표 함수
def evaluate(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch["labels"].cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='binary')  # binary classification
    return total_loss / len(data_loader), acc, f1

# ✅ 훈련 함수
def train_epoch(model, data_loader):
    model.train()
    total_loss = 0
    for batch in tqdm(data_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(data_loader)

# ✅ 전체 학습 루프
best_f1 = 0
start_time = time.time()

for epoch in range(3):
    print(f"\n🔁 Epoch {epoch+1} 시작")
    train_loss = train_epoch(model, train_loader)
    val_loss, val_acc, val_f1 = evaluate(model, test_loader)
    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}")

    # ✅ 가장 좋은 모델 저장
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), "best_bucketing_model.pt")
        print(f"✅ Best model saved (epoch {epoch+1})")

end_time = time.time()

# ✅ 학습 시간 출력
elapsed = end_time - start_time
hours, minutes, seconds = int(elapsed // 3600), int((elapsed % 3600) // 60), int(elapsed % 60)
print(f"\n🕒 총 학습 시간: {hours}시간 {minutes}분 {seconds}초")

# ✅ 테스트 평가
print("\n📊 [Test] Evaluating best model...")
test_start_time = time.time()

model.load_state_dict(torch.load("best_bucketing_model.pt"))
test_loss, test_acc, test_f1 = evaluate(model, test_loader)

test_end_time = time.time()
test_duration = test_end_time - test_start_time

print(f"[Test] Loss: {test_loss:.4f} | Acc: {test_acc:.4f} | F1: {test_f1:.4f}")
print(f"🕒 Test Evaluation Time: {test_duration:.2f} seconds")


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized


🔁 Epoch 1 시작


100%|██████████| 9375/9375 [18:01<00:00,  8.67it/s]


[Epoch 1] Train Loss: 0.2739, Val Loss: 0.2474, Acc: 0.9002, F1: 0.9030
✅ Best model saved (epoch 1)

🔁 Epoch 2 시작


100%|██████████| 9375/9375 [18:01<00:00,  8.67it/s]


[Epoch 2] Train Loss: 0.1844, Val Loss: 0.2446, Acc: 0.9044, F1: 0.9036
✅ Best model saved (epoch 2)

🔁 Epoch 3 시작


100%|██████████| 9375/9375 [18:01<00:00,  8.67it/s]


[Epoch 3] Train Loss: 0.1215, Val Loss: 0.2773, Acc: 0.8995, F1: 0.9033

🕒 총 학습 시간: 0시간 58분 48초

📊 [Test] Evaluating best model...
[Test] Loss: 0.2446 | Acc: 0.9044 | F1: 0.9036
🕒 Test Evaluation Time: 89.72 seconds


## 📊 모델 성능 비교

| 항목               | 이전 모델 (Trainer) | 현재 모델 (Custom loop + Bucketing) |
|--------------------|----------------------|--------------------------------------|
| **Test Loss**      | 0.4660               | 0.2446                               |
| **Test Accuracy**  | 0.8601               | 0.9044                               |
| **Test F1 Score**  | 0.8540               | 0.9036                               |


## 훈련 시간 비교
| 항목                | 이전 모델 (Trainer) | 현재 모델 (Custom loop + Bucketing)     |
| ----------------- | --------------- | ----------------------------------- |
| **총 Epoch 수**     | 3            | 3| 
| **총 훈련 시간**       | 3시간 42분      |    58분 48초                 

bucketing의 효과
- 모델 성능을 비교하였을 때 이전 모델보다 성능이 개선됨(accuracy, f1)
- 시간효율적 측면에서 훈련시간이 3시간 42분에서 58분 48초로 75%감소
- 동적패딩 + bucketing 전략을 사용하면 훈련속도와 모델 성능 모두에 긍정적인 영향을 준다고 할 수 있다. 

# 회고
- 학습 도중 OOM(Out of Memory) 에러가 빈번하게 발생했으나, 모델 학습 및 저장 후 GPU메모리를 명시적으로 초기화함으로써 문제를 해결할 수 있었다.
- 메모리 최적화 루틴(gc.collect() + torch.cuda.empty_cache())의 중요성을 경험했다. 
- basemodel 학습 시 validation loss가 점차 증가하며 과적합 양상을 보였다.
- dropout 적용, learning rate 조정, early stopping 등을 적용하여 validation loss가 감소하는 것을 확인하였지만 accuracy는 크게 향상되지 않음(0.9026 -> 0.9032)
- epoch은 3을 넘지 않아야 과적합을 방지할 수 있었다. 